In [36]:
%matplotlib inline
from selenium import webdriver
from pandas import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
import datetime
from bs4 import BeautifulSoup as bs
import requests as req
from urllib.request import urlopen
from lxml import etree
path = os.getcwd()

In [32]:
all_files = glob.glob(path + '/data' + "/schedule*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [33]:
frame.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,Wed Feb 1 2012,7:00p,Washington Wizards,103,Orlando Magic,109,Box Score,NaN,18846,NaN
1,Wed Feb 1 2012,7:00p,Chicago Bulls,82,Philadelphia 76ers,98,Box Score,NaN,18325,NaN
2,Wed Feb 1 2012,7:30p,Toronto Raptors,64,Boston Celtics,100,Box Score,NaN,18624,NaN
3,Wed Feb 1 2012,7:30p,Detroit Pistons,96,New Jersey Nets,99,Box Score,NaN,10504,NaN
4,Wed Feb 1 2012,8:00p,Oklahoma City Thunder,95,Dallas Mavericks,86,Box Score,NaN,20316,NaN


In [58]:
name = pd.read_csv(path + '/data/name_abb.csv')
name_dict = dict(zip(name['Franchise'],name['Abbreviation/Acronym']))
teamid_lookup = pd.read_json(path + '/data/' + 'nba_teamid.json')
TeamID = dict(zip(teamid_lookup['abbreviation'], teamid_lookup['teamId']))

In [59]:
s_team_sorted = frame.sort_values(by = ['Home/Neutral'])

In [60]:
s_team_sorted['Date_c'] = s_team_sorted['Date'].str[4:]

In [62]:
s_team_sorted['Home_abb'] = [name_dict[n] for n in s_team_sorted['Home/Neutral']]

In [63]:
datelist = []
for date in s_team_sorted['Date_c']:
    datelist.append(datetime.datetime.strptime(date, '%b %d %Y'))

In [64]:
s_team_sorted['Date_fmt'] = datelist
s_team_sorted['Date_shifted'] = s_team_sorted['Date_fmt'] + datetime.timedelta(days=1)
s_team_sorted['Date_shifted'] = [datetime.datetime.strftime(d, "%m-%d-%Y") for d in s_team_sorted['Date_shifted']]

In [65]:
teamid_lookup = pd.read_json(path + '/data/' + 'nba_teamid.json')

In [66]:
#add season
season = []
for gameday in s_team_sorted['Date_fmt']:
    if datetime.date(2006,10,1) <= gameday.date() <= datetime.date(2007,7,1):
        season.append('2006-07')
    elif datetime.date(2007,10,1) <= gameday.date() <= datetime.date(2008,7,1):
        season.append('2007-08')
    elif datetime.date(2008,10,1) <= gameday.date() <= datetime.date(2009,7,1):
        season.append('2008-09')
    elif datetime.date(2009,10,1) <= gameday.date() <= datetime.date(2010,7,1):
        season.append('2009-10')
    elif datetime.date(2010,10,1) <= gameday.date() <= datetime.date(2011,7,1):
        season.append('2010-11')
    elif datetime.date(2011,10,1) <= gameday.date() <= datetime.date(2012,7,1):
        season.append('2011-12')        
    elif datetime.date(2012,10,1) <= gameday.date() <= datetime.date(2013,7,1):
        season.append('2012-13')       
    elif datetime.date(2013,10,1) <= gameday.date() <= datetime.date(2014,7,1):
        season.append('2013-14')    
    elif datetime.date(2014,10,1) <= gameday.date() <= datetime.date(2015,7,1):
        season.append('2014-15')
    elif datetime.date(2015,10,1) <= gameday.date() <= datetime.date(2016,7,1):
        season.append('2015-16')     
    elif datetime.date(2016,10,1) <= gameday.date() <= datetime.date(2017,7,1):
        season.append('2016-17')  
    elif datetime.date(2017,10,1) <= gameday.date() <= datetime.date(2018,7,1):
        season.append('2017-18')
    elif datetime.date(2018,10,1) <= gameday.date() <= datetime.date(2019,7,1):
        season.append('2018-19')
s_team_sorted['Season'] = season

In [67]:
year = ['2007-08','2008-09','2009-10','2010-11','2011-12','2012-13','2013-14','2014-15','2015-16',
       '2016-17','2017-18']
TeamID = dict(zip(teamid_lookup['abbreviation'], teamid_lookup['teamId']))

# Using the ChromeDriver: too slow


#finding them xpath

season year:
/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[1]

Hawks:
/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[4]/div/div/label/select/option[1]
/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[4]/div/div/label/select/option[2]

vs Celtics:
driver.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[2]/div[1]/div[1]/div[2]/div/label/select/option[3]').click()
/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select

date:
/html/body/main/div[2]/div/div[2]/div/div/div[2]/div[1]/div[3]/div[6]/div/input
option to all:
/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[3]/div/div/select
(option[2] for 2015-2016 season)

In [14]:
# teamsnb = range(1,31)
# driver.get(url)
# #season year:
# driver.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[12]').click()
# #Hawks
# driver.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[4]/div/div/label/select/option[2]').click()
# # #advanced filter
# driver.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[6]/a').click()
# # set the date
# driver.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[2]/div[1]/div[3]/div[6]/div/input[@value =str(Nov 14 2007)]')
# #run it
# driver.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[2]/div[2]/stats-run-it/a').click()
# #option to all
# driver.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]').click()

In [15]:
# #find the table
# table = driver.find_element_by_class_name('nba-stat-table__overflow')

In [16]:
# lineups = []
# team = []
# stats = []

# for line_id, lines in enumerate(table.text.split('\n')):
#     if line_id == 0:
#          columns_names = lines.split(' ')[1:]
#          print(columns_names)
#     else:
#         line = lines.split('ATL')
#         lineups.append(line[0])
#         line_stats = [i for i in line[1].split(' ')][1:]
#         stats.append([float(s) for s in line_stats])
# #         team.append(lines.split(',')[6])
# #         stats.append([lines.split(',')[7:]])

In [17]:
db = pd.DataFrame({'lineup':lineups,
                   'GP':[i[0] for i in stats],
                   'MIN':[i[1] for i in stats],
                   'PTS':[i[2] for i in stats],
                   'FGM':[i[3] for i in stats],
                   'FGA':[i[4] for i in stats],
                   'FG%':[i[5] for i in stats],
                   '3PM':[i[6] for i in stats],
                   '3PA':[i[7] for i in stats],
                   '3P%':[i[8] for i in stats],
                   'FTM':[i[9] for i in stats],
                   'FTA':[i[10] for i in stats],
                   'FT%':[i[11] for i in stats],
                   'OREB':[i[12] for i in stats],
                   'DREB':[i[13] for i in stats],
                   'REB':[i[14] for i in stats],
                   'AST':[i[15] for i in stats],
                   'TOV':[i[16] for i in stats],
                   'STL':[i[17] for i in stats],
                   'BLK':[i[18] for i in stats],
                   'BLKA':[i[19] for i in stats],
                   'PF':[i[20] for i in stats],
                   'PFD':[i[21] for i in stats],
                   '+/-':[i[22] for i in stats]
                    })

NameError: name 'lineups' is not defined

# parse ATL 2007-2008 All Game Lineup

The syntax for a link is:
"https://stats.nba.com/lineups/traditional/?Season=2007-08&SeasonType=Regular%20Season&TeamID=1610612737&DateFrom=Nov%2F04%2F2007&DateTo=11%2F05%2F2007"

In [14]:
#driving the browser
year = ['2007-08','2008-09','2009-10','2010-11','2011-12','2012-13','2013-14','2014-15','2015-16',
       '2016-17','2017-18']

  # Optional argument, if not specified will search path.

In [18]:
s_team_sorted_0708 = s_team_sorted[s_team_sorted.Season != '2006-07']

In [19]:
db_init.to_csv(index=True)

NameError: name 'db_init' is not defined

In [20]:
# url = 'https://stats.nba.com/lineups/traditional/?Season='+season+'&SeasonType=Regular%20Season&TeamID='+teamid+date
driver = webdriver.Chrome('/Users/yezichang/Desktop/NYU/1001/Project/chromedriver')
track = 0
import time
import random

for index, row in s_team_sorted_0708.iterrows():
    
    track += 1
    
    # pauase the program for a while
    pause = 5*(random.random())
    print('THIS IS NUMBER ', track)
    year = row['Season']
    teamid = str(TeamID[row['Home_abb']])
    date = '&DateFrom='+str(row['Date_c']).replace(' ','%2F')+'&DateTo='+str(row['Date_shifted']).replace('-','%2F')
    
    url = 'https://stats.nba.com/lineups/traditional/?Season='+year+'&TeamID='+teamid+date
    print("Request #%d"%track)
    
    try:
        driver.get(url)
        table = driver.find_element_by_class_name("nba-stat-table")
        if track == 1:
            db_init = statsParser(table,team = row['Home_abb'] ,gameday = row['Date_c'])
            print(db_init.shape)
        else:
            db_new = statsParser(table,team = row['Home_abb'] ,gameday = row['Date_c'])
            db_init = pd.concat([db_init,db_new],axis = 0)
            print(db_init.shape)
    except:
        db_init = db_init
        print('Error!')
        print(row['Home_abb'])
        print(url)
        print(row['Date_c'])
    
    time.sleep(pause)

KeyboardInterrupt: 

# Parsing Traditional Data From 07-19

- Issues:
    - we need the old team abbreviation in the parsing step, so I change the team lookup to its original style
        - Seattle SuperSonics - SEA, sharing the same teamId as OKC
        - Charlotte Bobcats to Charlotte Hornets, sharing the same teamId as CHA
        - New Orleans Hornets to New Orleans Pelicans, sharing the same teamID as NOP
    - missing the offseason
    - webdriver tries next page after a few second - fast and stable internets needed
        

In [68]:
# get the data ready
s_07_19 = s_team_sorted[s_team_sorted.Season != '2006-07']
s_07_19 = s_07_19.sort_values(by = ['Season'])

I tried to use proxy, but it was not stable. I ended up using VPN.

In [15]:
import requests
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:1000]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [16]:
db_init = pd.DataFrame({'lineup':[],
                   'GP':[],
                   'MIN':[],
                   'PTS':[],
                   'FGM':[],
                   'FGA':[],
                   'FG%':[],
                   '3PM':[],
                   '3PA':[],
                   '3P%':[],
                   'FTM':[],
                   'FTA':[],
                   'FT%':[],
                   'OREB':[],
                   'DREB':[],
                   'REB':[],
                   'AST':[],
                   'TOV':[],
                   'STL':[],
                   'BLK':[],
                   'BLKA':[],
                   'PF':[],
                   'PFD':[],
                   '+/-':[],
                   'TEAM':[],
                   'GAMEDAY':[]
                    })

In [30]:
# try to get the url using different id

def urldrivget(url):
    proxs = get_proxies()
    options = Options()
    # goal is to find the working proxy
    # and be able to switch once it is disconnected
    options.add_argument('--proxy-server=199.167.137.234')
    driver = webdriver.Chrome('/Users/yezichang/Desktop/NYU/1001/Project/chromedriver', chrome_options = options)
    driver.get(url)
    time.sleep(3)
    table = driver.find_element_by_class_name("nba-stat-table") 
    return table


In [3]:
# we want the traditional data
track = 0
import time
import random
# goal is to find the working proxy
# and be able to switch once it is disconnected
db_init = pd.DataFrame({'lineup':[],
                   'GP':[],
                   'MIN':[],
                   'PTS':[],
                   'FGM':[],
                   'FGA':[],
                   'FG%':[],
                   '3PM':[],
                   '3PA':[],
                   '3P%':[],
                   'FTM':[],
                   'FTA':[],
                   'FT%':[],
                   'OREB':[],
                   'DREB':[],
                   'REB':[],
                   'AST':[],
                   'TOV':[],
                   'STL':[],
                   'BLK':[],
                   'BLKA':[],
                   'PF':[],
                   'PFD':[],
                   '+/-':[],
                   'TEAM':[],
                   'GAMEDAY':[]
                    })
for index, row in s_07_19.iterrows():
    
    track += 1
    
    try:
        # pauase the program for a while
        pause = 5*(random.random())
        print('THIS IS NUMBER ', track)
        year = row['Season']
        teamid = str(TeamID[row['Home_abb']])
        date = '&DateFrom='+str(row['Date_c']).replace(' ','%2F')+'&DateTo='+str(row['Date_shifted']).replace('-','%2F')

        url = 'https://stats.nba.com/lineups/traditional/?Season='+year+'&TeamID='+teamid+date
        print("Request #%d"%track)

        try:
            # goal is to find the working proxy
            # and be able to switch once it is disconnected
            table = driver.find_element_by_class_name('nba-stat-table') 
            if track == 1:
                db_init = statsParser(table,team = row['Home_abb'] ,gameday = row['Date_c'])
                print(db_init.shape)ß
            else:
                db_new = statsParser(table,team = row['Home_abb'] ,gameday = row['Date_c'])
                db_init = pd.concat([db_init,db_new],axis = 0)
                print(db_init.shape)
        except:
            db_init = db_init
            print('Error!')
            print(row['Home_abb'])
            print(url)
            print(row['Date_c'])
        time.sleep(pause)

    # this allows me pause the program
    except KeyboardInterrupt:
        print ('\nPausing...  (Hit ENTER to continue, type quit to exit.)')
        try:
            response = input()
            if response == 'quit':
                break
            print ('Resuming...')
        except KeyboardInterrupt:
            print ('Resuming...')
            continue

NameError: name 'Options' is not defined

In [166]:
db_init.to_csv(path+'/data/lineup_traditional_0719_m.csv')

In [90]:
db_init.head() # results table

,lineup,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-,TEAM,GAMEDAY
0,".B. Barry, .M. Finley, .T. Duncan, .M. Ginobil...",1,10.1,26.0,12.0,20.0,60.0,2.0,6.0,33.3,...,7.0,2.0,2.0,0.0,1.0,1.0,3.0,10.0,SAS,Nov 16 2007
1,".M. Finley, .B. Bowen, .T. Duncan, .T. Parker,...",1,9.9,16.0,7.0,13.0,53.8,1.0,1.0,100,...,3.0,3.0,1.0,0.0,0.0,6.0,3.0,3.0,SAS,Nov 16 2007
2,".B. Bowen, .T. Duncan, .F. Elson, .M. Ginobili...",1,4.1,10.0,3.0,7.0,42.9,0.0,1.0,0.0,...,2.0,1.0,1.0,0.0,2.0,0.0,3.0,4.0,SAS,Nov 16 2007
3,".M. Finley, .J. Vaughn, .F. Elson, .M. Ginobil...",1,3.2,3.0,1.0,3.0,33.3,1.0,1.0,100,...,0.0,4.0,1.0,0.0,0.0,2.0,1.0,-1.0,SAS,Nov 16 2007
4,".B. Barry, .M. Finley, .J. Vaughn, .F. Elson, ...",1,2.9,9.0,3.0,4.0,75.0,1.0,1.0,100,...,2.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,SAS,Nov 16 2007


# Error Label


In [65]:
import os

def readerr(file):
    lines = file.readlines()
    errorlabel = [0] * 15625 # number of instances attempted

    for i in range(len(lines)):
        if 'Error!' in lines[i]:
            tar = int(lines[i-1][9:]) # index of instance that went wrong
            errorlabel[tar-1] = 1
        else:
            continue

    return errorlabel


# path = os.getcwd()
# file = open(path + '/data/parsing_output_07-19.txt')
# label = readerr(file)

In [70]:
s_07_19['Errorlabel'] = label

In [71]:
# we have about 17.2% of the game
15625 - np.sum(s_07_19['Errorlabel'])

2690

# Label the regular season

In [72]:
regularseason = []
for gameday in s_07_19['Date_fmt']:
    if datetime.date(2006,10,31) <= gameday.date() <= datetime.date(2007,4,18):
        regularseason.append(1)
    elif datetime.date(2007,10,30) <= gameday.date() <= datetime.date(2008,4,16):
        regularseason.append(1)
    elif datetime.date(2008,10,28) <= gameday.date() <= datetime.date(2009,4,16):
        regularseason.append(1)
    elif datetime.date(2009,10,27) <= gameday.date() <= datetime.date(2010,4,14):
        regularseason.append(1)
    elif datetime.date(2010,10,26) <= gameday.date() <= datetime.date(2011,4,13):
        regularseason.append(1)
    elif datetime.date(2011,12,25) <= gameday.date() <= datetime.date(2012,4,26):
        regularseason.append(1)        
    elif datetime.date(2012,10,30) <= gameday.date() <= datetime.date(2013,4,17):
        regularseason.append(1)    
    elif datetime.date(2013,10,29) <= gameday.date() <= datetime.date(2014,4,16):
        regularseason.append(1)  
    elif datetime.date(2014,10,28) <= gameday.date() <= datetime.date(2015,4,15):
        regularseason.append(1)
    elif datetime.date(2015,10,27) <= gameday.date() <= datetime.date(2016,4,13):
        regularseason.append(1)   
    elif datetime.date(2016,10,25) <= gameday.date() <= datetime.date(2017,4,12):
        regularseason.append(1)
    elif datetime.date(2017,10,17) <= gameday.date() <= datetime.date(2018,4,11):
        regularseason.append(1)
    elif datetime.date(2018,10,16) <= gameday.date() <= datetime.date(2019,4,10):
        regularseason.append(1)
    else:
        regularseason.append(0)
s_07_19['RegularSeason'] = regularseason

In [73]:
s_07_19.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes,Date_c,Home_abb,Date_fmt,Date_shifted,Season,Errorlabel,RegularSeason
8440,Mon Jan 14 2008,10:00p,Los Angeles Lakers,123,Seattle SuperSonics,121,Box Score,OT,13452,NaN,Jan 14 2008,SEA,2008-01-14,01-15-2008,2007-08,0,1
15213,Fri Nov 16 2007,8:00p,Houston Rockets,84,San Antonio Spurs,90,Box Score,NaN,18797,NaN,Nov 16 2007,SAS,2007-11-16,11-17-2007,2007-08,0,1
16531,Wed Dec 12 2007,7:30p,Sacramento Kings,78,Boston Celtics,90,Box Score,NaN,18624,NaN,Dec 12 2007,BOS,2007-12-12,12-13-2007,2007-08,0,1
16605,Sat Dec 22 2007,7:30p,Utah Jazz,102,Miami Heat,104,Box Score,NaN,19600,NaN,Dec 22 2007,MIA,2007-12-22,12-23-2007,2007-08,0,1
3059,Wed May 28 2008,8:30p,Detroit Pistons,102,Boston Celtics,106,Box Score,NaN,18624,NaN,May 28 2008,BOS,2008-05-28,05-29-2008,2007-08,1,0


In [74]:
s_07_19.to_csv(path+'/data/s_07_19_m.csv')

# Parsing 2.0 07-19 Regular Seasons
- Let's focus on the error, missing ones
- why does the statsparser have an array length problem?
    - naming convention, New Jersey Nets' abb is NJN
    

In [2]:
# setting up
name = pd.read_csv(path + '/data/name_abb.csv')
name_dict = dict(zip(name['Franchise'],name['Abbreviation/Acronym']))
teamid_lookup = pd.read_json(path + '/data/' + 'nba_teamid.json')
TeamID = dict(zip(teamid_lookup['abbreviation'], teamid_lookup['teamId']))

In [3]:
# lets focus on the error ones:
again = pd.read_csv(path+'/data/s_07_19_m.csv')
again = again.loc[(again.Errorlabel == 1) & (again.RegularSeason == 1)]

In [4]:
again.shape

(11946, 18)

In [5]:
again.Home_abb.unique()

array(['CLE', 'MIA', 'NOH', 'SAC', 'BOS', 'NJN', 'MIN', 'CHA', 'DAL',
       'PHX', 'MIL', 'POR', 'PHI', 'IND', 'CHI', 'UTA', 'NYK', 'HOU',
       'ORL', 'DET', 'GSW', 'ATL', 'LAC', 'WAS', 'DEN', 'LAL', 'TOR',
       'OKC', 'MEM', 'SAS', 'BKN', 'NOP'], dtype=object)

In [6]:
def statsParser(table,team,gameday):
    lineups = []
    stats = []
    for line_id, lines in enumerate(table.text.split('\n')):
        if line_id == 0:
             columns_names = lines.split(' ')[1:]
        elif('LINEUPS' in lines) & (line_id != 0):
            break
        else:
            line = lines.split(team)
            lineups.append(line[0])
            try:
                stats.append(line[1].split(' ')[1:])
            except:
                continue
    teamls = [team for i in range(len(stats))]
    gamedayls = [gameday for i in range(len(stats))]
    db = pd.DataFrame({'lineup':lineups,
                   'GP':[i[0] for i in stats],
                   'MIN':[i[1] for i in stats],
                   'PTS':[i[2] for i in stats],
                   'FGM':[i[3] for i in stats],
                   'FGA':[i[4] for i in stats],
                   'FG%':[i[5] for i in stats],
                   '3PM':[i[6] for i in stats],
                   '3PA':[i[7] for i in stats],
                   '3P%':[i[8] for i in stats],
                   'FTM':[i[9] for i in stats],
                   'FTA':[i[10] for i in stats],
                   'FT%':[i[11] for i in stats],
                   'OREB':[i[12] for i in stats],
                   'DREB':[i[13] for i in stats],
                   'REB':[i[14] for i in stats],
                   'AST':[i[15] for i in stats],
                   'TOV':[i[16] for i in stats],
                   'STL':[i[17] for i in stats],
                   'BLK':[i[18] for i in stats],
                   'BLKA':[i[19] for i in stats],
                   'PF':[i[20] for i in stats],
                   'PFD':[i[21] for i in stats],
                   '+/-':[i[22] for i in stats],
                   'TEAM':teamls,
                   'GAMEDAY':gamedayls
                    })
    return db

In [27]:
# we want the traditional data
import time
import random


start = init = 10095
options2 = webdriver.ChromeOptions()
driver = webdriver.Chrome('/Users/yezichang/Desktop/NYU/1001/Project/chromedriver')
end = 11946
db_init = pd.DataFrame({'lineup':[],
                   'GP':[],
                   'MIN':[],
                   'PTS':[],
                   'FGM':[],
                   'FGA':[],
                   'FG%':[],
                   '3PM':[],
                   '3PA':[],
                   '3P%':[],
                   'FTM':[],
                   'FTA':[],
                   'FT%':[],
                   'OREB':[],
                   'DREB':[],
                   'REB':[],
                   'AST':[],
                   'TOV':[],
                   'STL':[],
                   'BLK':[],
                   'BLKA':[],
                   'PF':[],
                   'PFD':[],
                   '+/-':[],
                   'TEAM':[],
                   'GAMEDAY':[]
                    })

for index, row in again[start:end].iterrows():

    start += 1
    try:
        # pauase the program for a while
        pause = 5*(random.random())
        print('THIS IS NUMBER ', start)
        year = row['Season']
        teamid = str(TeamID[row['Home_abb']])
        date = '&DateFrom='+str(row['Date_c']).replace(' ','%2F')+'&DateTo='+str(row['Date_shifted']).replace('-','%2F')

        url = 'https://stats.nba.com/lineups/traditional/?Season='+year+'&TeamID='+teamid+date
        print("Request #%d"%start)

        try:
            driver.get(url)
            time.sleep(2)
            table = driver.find_element_by_class_name("nba-stat-table")
            if start == 1:
                db_init = statsParser(table,team = row['Home_abb'] ,gameday = row['Date_c'])
                print(db_init.shape)
            else:
                db_new = statsParser(table,team = row['Home_abb'] ,gameday = row['Date_c'])
                db_init = pd.concat([db_init,db_new],axis = 0)
                print(db_init.shape)
        except:
            db_init = db_init
            print('Error!')
            print(row['Home_abb'])
            print(url)
            print(row['Date_c'])
        time.sleep(pause)

    # this allows me pause the program
    except KeyboardInterrupt:
        print ('\nPausing...  (Hit ENTER to continue, type quit to exit.)')
        try:
            response = input()
            if response == 'quit':
                break
            print ('Resuming...')
        except KeyboardInterrupt:
            print ('Resuming...')
            continue

THIS IS NUMBER  10096
Request #10096
(15, 26)
THIS IS NUMBER  10097
Request #10097
(31, 26)
THIS IS NUMBER  10098
Request #10098
(53, 26)
THIS IS NUMBER  10099
Request #10099
(70, 26)
THIS IS NUMBER  10100
Request #10100
(83, 26)
THIS IS NUMBER  10101
Request #10101
(106, 26)
THIS IS NUMBER  10102
Request #10102
(129, 26)
THIS IS NUMBER  10103
Request #10103
(142, 26)
THIS IS NUMBER  10104
Request #10104
(161, 26)
THIS IS NUMBER  10105
Request #10105
(178, 26)
THIS IS NUMBER  10106
Request #10106
(203, 26)
THIS IS NUMBER  10107
Request #10107
(225, 26)
THIS IS NUMBER  10108
Request #10108
(250, 26)
THIS IS NUMBER  10109
Request #10109
(275, 26)
THIS IS NUMBER  10110
Request #10110
(295, 26)
THIS IS NUMBER  10111
Request #10111
(312, 26)
THIS IS NUMBER  10112
Request #10112
(337, 26)
THIS IS NUMBER  10113
Request #10113
(362, 26)
THIS IS NUMBER  10114
Request #10114
(387, 26)
THIS IS NUMBER  10115
Request #10115
(405, 26)
THIS IS NUMBER  10116
Request #10116
(417, 26)
THIS IS NUMBER  10

(3158, 26)
THIS IS NUMBER  10260
Request #10260
(3177, 26)
THIS IS NUMBER  10261
Request #10261
(3198, 26)
THIS IS NUMBER  10262
Request #10262
(3223, 26)
THIS IS NUMBER  10263
Request #10263
(3238, 26)
THIS IS NUMBER  10264
Request #10264
(3258, 26)
THIS IS NUMBER  10265
Request #10265
(3281, 26)
THIS IS NUMBER  10266
Request #10266
(3298, 26)
THIS IS NUMBER  10267
Request #10267
(3318, 26)
THIS IS NUMBER  10268
Request #10268
(3343, 26)
THIS IS NUMBER  10269
Request #10269
(3365, 26)
THIS IS NUMBER  10270
Request #10270
(3384, 26)
THIS IS NUMBER  10271
Request #10271
(3404, 26)
THIS IS NUMBER  10272
Request #10272
(3426, 26)
THIS IS NUMBER  10273
Request #10273
(3441, 26)
THIS IS NUMBER  10274
Request #10274
(3459, 26)
THIS IS NUMBER  10275
Request #10275
(3477, 26)
THIS IS NUMBER  10276
Request #10276
(3501, 26)
THIS IS NUMBER  10277
Request #10277
(3519, 26)
THIS IS NUMBER  10278
Request #10278
(3536, 26)
THIS IS NUMBER  10279
Request #10279
(3556, 26)
THIS IS NUMBER  10280
Request

(6480, 26)
THIS IS NUMBER  10428
Request #10428
(6500, 26)
THIS IS NUMBER  10429
Request #10429
(6524, 26)
THIS IS NUMBER  10430
Request #10430
(6544, 26)
THIS IS NUMBER  10431
Request #10431
(6564, 26)
THIS IS NUMBER  10432
Request #10432
(6576, 26)
THIS IS NUMBER  10433
Request #10433
(6588, 26)
THIS IS NUMBER  10434
Request #10434
(6613, 26)
THIS IS NUMBER  10435
Request #10435
(6631, 26)
THIS IS NUMBER  10436
Request #10436
(6652, 26)
THIS IS NUMBER  10437
Request #10437
(6673, 26)
THIS IS NUMBER  10438
Request #10438
(6698, 26)
THIS IS NUMBER  10439
Request #10439
(6716, 26)
THIS IS NUMBER  10440
Request #10440
(6734, 26)
THIS IS NUMBER  10441
Request #10441
(6758, 26)
THIS IS NUMBER  10442
Request #10442
(6773, 26)
THIS IS NUMBER  10443
Request #10443
(6794, 26)
THIS IS NUMBER  10444
Request #10444
(6809, 26)
THIS IS NUMBER  10445
Request #10445
(6833, 26)
THIS IS NUMBER  10446
Request #10446
(6851, 26)
THIS IS NUMBER  10447
Request #10447
(6871, 26)
THIS IS NUMBER  10448
Request

THIS IS NUMBER  10586
Request #10586
(9658, 26)
THIS IS NUMBER  10587
Request #10587
(9683, 26)
THIS IS NUMBER  10588
Request #10588
(9708, 26)
THIS IS NUMBER  10589
Request #10589
(9726, 26)
THIS IS NUMBER  10590
Request #10590
(9751, 26)
THIS IS NUMBER  10591
Request #10591
(9767, 26)
THIS IS NUMBER  10592
Request #10592
(9788, 26)
THIS IS NUMBER  10593
Request #10593
(9812, 26)
THIS IS NUMBER  10594
Request #10594
(9836, 26)
THIS IS NUMBER  10595
Request #10595
(9855, 26)
THIS IS NUMBER  10596
Request #10596
(9872, 26)
THIS IS NUMBER  10597
Request #10597
(9879, 26)
THIS IS NUMBER  10598
Request #10598
(9904, 26)
THIS IS NUMBER  10599
Request #10599
(9929, 26)
THIS IS NUMBER  10600
Request #10600
(9951, 26)
THIS IS NUMBER  10601
Request #10601
(9971, 26)
THIS IS NUMBER  10602
Request #10602
(9996, 26)
THIS IS NUMBER  10603
Request #10603
(10011, 26)
THIS IS NUMBER  10604
Request #10604
(10036, 26)
THIS IS NUMBER  10605
Request #10605
(10054, 26)
THIS IS NUMBER  10606
Request #10606


(12840, 26)
THIS IS NUMBER  10743
Request #10743
(12862, 26)
THIS IS NUMBER  10744
Request #10744
(12884, 26)
THIS IS NUMBER  10745
Request #10745
(12906, 26)
THIS IS NUMBER  10746
Request #10746
(12928, 26)
THIS IS NUMBER  10747
Request #10747
(12949, 26)
THIS IS NUMBER  10748
Request #10748
(12965, 26)
THIS IS NUMBER  10749
Request #10749
(12985, 26)
THIS IS NUMBER  10750
Request #10750
(13007, 26)
THIS IS NUMBER  10751
Request #10751
(13032, 26)
THIS IS NUMBER  10752
Request #10752
(13055, 26)
THIS IS NUMBER  10753
Request #10753
(13074, 26)
THIS IS NUMBER  10754
Request #10754
(13089, 26)
THIS IS NUMBER  10755
Request #10755
(13109, 26)
THIS IS NUMBER  10756
Request #10756
(13127, 26)
THIS IS NUMBER  10757
Request #10757
(13150, 26)
THIS IS NUMBER  10758
Request #10758
(13164, 26)
THIS IS NUMBER  10759
Request #10759
(13181, 26)
THIS IS NUMBER  10760
Request #10760
(13203, 26)
THIS IS NUMBER  10761
Request #10761
(13219, 26)
THIS IS NUMBER  10762
Request #10762
(13235, 26)
THIS IS 

...

THIS IS NUMBER  11493
Request #11493
Error!
DEN
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612743&DateFrom=Nov%2F9%2F2018&DateTo=11%2F10%2F2018
Nov 9 2018
THIS IS NUMBER  11494
Request #11494
Error!
PHI
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612755&DateFrom=Nov%2F28%2F2018&DateTo=11%2F29%2F2018
Nov 28 2018
THIS IS NUMBER  11495
Request #11495
Error!
PHI
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612755&DateFrom=Feb%2F8%2F2019&DateTo=02%2F09%2F2019
Feb 8 2019
THIS IS NUMBER  11496
Request #11496
Error!
SAS
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612759&DateFrom=Oct%2F17%2F2018&DateTo=10%2F18%2F2018
Oct 17 2018
THIS IS NUMBER  11497
Request #11497
Error!
MEM
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612763&DateFrom=Feb%2F27%2F2019&DateTo=02%2F28%2F2019
Feb 27 2019
THIS IS NUMBER  11498
Request #11498
Error!
SAS
https://stats.nba.com/lineups/traditional

Error!
NYK
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612752&DateFrom=Feb%2F5%2F2019&DateTo=02%2F06%2F2019
Feb 5 2019
THIS IS NUMBER  11541
Request #11541
Error!
PHI
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612755&DateFrom=Oct%2F27%2F2018&DateTo=10%2F28%2F2018
Oct 27 2018
THIS IS NUMBER  11542
Request #11542
Error!
BOS
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612738&DateFrom=Nov%2F21%2F2018&DateTo=11%2F22%2F2018
Nov 21 2018
THIS IS NUMBER  11543
Request #11543
Error!
DEN
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612743&DateFrom=Mar%2F2%2F2019&DateTo=03%2F03%2F2019
Mar 2 2019
THIS IS NUMBER  11544
Request #11544
Error!
MEM
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612763&DateFrom=Dec%2F29%2F2018&DateTo=12%2F30%2F2018
Dec 29 2018
THIS IS NUMBER  11545
Request #11545
Error!
PHI
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612755&Da

Error!
BOS
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612738&DateFrom=Feb%2F9%2F2019&DateTo=02%2F10%2F2019
Feb 9 2019
THIS IS NUMBER  11586
Request #11586
Error!
DEN
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612743&DateFrom=Feb%2F1%2F2019&DateTo=02%2F02%2F2019
Feb 1 2019
THIS IS NUMBER  11587
Request #11587
Error!
DEN
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612743&DateFrom=Jan%2F13%2F2019&DateTo=01%2F14%2F2019
Jan 13 2019
THIS IS NUMBER  11588
Request #11588
Error!
NOP
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612740&DateFrom=Dec%2F7%2F2018&DateTo=12%2F08%2F2018
Dec 7 2018
THIS IS NUMBER  11589
Request #11589
Error!
DEN
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612743&DateFrom=Jan%2F17%2F2019&DateTo=01%2F18%2F2019
Jan 17 2019
THIS IS NUMBER  11590
Request #11590
Error!
NOP
https://stats.nba.com/lineups/traditional/?Season=2018-19&TeamID=1610612740&Date

THIS IS NUMBER  11641
Request #11641
(20216, 26)
THIS IS NUMBER  11642
Request #11642
(20241, 26)
THIS IS NUMBER  11643
Request #11643
(20265, 26)
THIS IS NUMBER  11644
Request #11644
(20290, 26)
THIS IS NUMBER  11645
Request #11645
(20315, 26)
THIS IS NUMBER  11646
Request #11646
(20336, 26)
THIS IS NUMBER  11647
Request #11647
(20353, 26)
THIS IS NUMBER  11648
Request #11648
(20378, 26)
THIS IS NUMBER  11649
Request #11649
(20395, 26)
THIS IS NUMBER  11650
Request #11650
(20420, 26)
THIS IS NUMBER  11651
Request #11651
(20440, 26)
THIS IS NUMBER  11652
Request #11652
(20458, 26)
THIS IS NUMBER  11653
Request #11653
(20481, 26)
THIS IS NUMBER  11654
Request #11654
(20506, 26)
THIS IS NUMBER  11655
Request #11655
(20527, 26)
THIS IS NUMBER  11656
Request #11656
(20543, 26)
THIS IS NUMBER  11657
Request #11657
(20558, 26)
THIS IS NUMBER  11658
Request #11658
(20583, 26)
THIS IS NUMBER  11659
Request #11659
(20605, 26)
THIS IS NUMBER  11660
Request #11660
(20630, 26)
THIS IS NUMBER  1166

THIS IS NUMBER  11803
Request #11803
(23431, 26)
THIS IS NUMBER  11804
Request #11804
(23448, 26)
THIS IS NUMBER  11805
Request #11805
(23463, 26)
THIS IS NUMBER  11806
Request #11806
(23484, 26)
THIS IS NUMBER  11807
Request #11807
(23502, 26)
THIS IS NUMBER  11808
Request #11808
(23527, 26)
THIS IS NUMBER  11809
Request #11809
(23552, 26)
THIS IS NUMBER  11810
Request #11810
(23571, 26)
THIS IS NUMBER  11811
Request #11811
(23588, 26)
THIS IS NUMBER  11812
Request #11812
(23613, 26)
THIS IS NUMBER  11813
Request #11813
(23626, 26)
THIS IS NUMBER  11814
Request #11814
(23643, 26)
THIS IS NUMBER  11815
Request #11815
(23660, 26)
THIS IS NUMBER  11816
Request #11816
(23682, 26)
THIS IS NUMBER  11817
Request #11817
(23703, 26)
THIS IS NUMBER  11818
Request #11818
(23723, 26)
THIS IS NUMBER  11819
Request #11819
(23748, 26)
THIS IS NUMBER  11820
Request #11820
(23769, 26)
THIS IS NUMBER  11821
Request #11821
(23790, 26)
THIS IS NUMBER  11822
Request #11822
(23808, 26)
THIS IS NUMBER  1182

In [12]:
db_init

(1288, 26)

In [28]:
db_init.to_csv(path+'/data/lineup_traditional_0719_again_%(track)s_%(end)s.csv' %{"track":str(init), "end": str(start)})

In [58]:
again_files = glob.glob(path + '/data/' + 'lineup_traditional_0719_again_*.csv')

ls = []
for file in again_files:
    df = pd.read_csv(file, index_col=None, header=0)
    ls.append(df)

again_df = pd.concat(ls, axis = 0, ignore_index = True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [42]:
again_df.to_csv(path+'/data/lineup_traditional_0719_again_0_11946.csv')

In [53]:
temp = again_df.groupby(['TEAM','GAMEDAY'],as_index = False)

In [62]:
len(temp.size())

8640

In [63]:
(2690+8640)/15625

0.72512

In [67]:
file = open(path + '/data/parsing_output_07-19_again.txt')
label2 = readerr(file)

In [70]:
label22 = label2[0:3000] + ['na']*len(range(3001,5096)) + label2[5096:]

(11946, 18)

15624